In [9]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [10]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)
    skip = getattr(module, "skip", False)

    if skip:
        print("  Marked for skip")
        continue

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:09<00:00, 1005.43it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:376: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



{'basic': {'expected_sample_burst_size': 2.994125385183717,
           'expected_sample_data_size': 16.49177373722812,
           'expected_time_to_recover': 98.9006,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.16177859386090682,
             'bit_redundancy': 414.503,
             'ideal_bit_efficency': 0.19114992785907028,
             'ideal_bit_redundancy': 338.51964526495584,
             'ideal_packet_efficiency': 0.9557496392953514,
             'ideal_packet_redundancy': 67.70392905299117,
             'ideal_time_to_recover': 83.70392905299117,
             'ideal_transmitted_bits': 418.51964526495584,
             'packet_efficiency': 0.808892969304534,
             'packet_redundancy': 82.9006,
             'permeability': 0.846343996426626,
             'saturation': 16.0,
             'transmitted_bits': 494.503}}
Found implementation: anfext2
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:05<00:00, 1880.59it/s]


{'basic': {'expected_sample_burst_size': 2.9951557370486026,
           'expected_sample_data_size': 16.56109376251101,
           'expected_time_to_recover': 39.5527,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.40452358498914104,
             'bit_redundancy': 117.76350000000002,
             'ideal_bit_efficency': 0.4776836765704345,
             'ideal_bit_redundancy': 87.47484564338885,
             'ideal_packet_efficiency': 2.388418382852173,
             'ideal_packet_redundancy': 17.494969128677766,
             'ideal_time_to_recover': 33.494969128677766,
             'ideal_transmitted_bits': 167.47484564338885,
             'packet_efficiency': 2.0226179249457052,
             'packet_redundancy': 23.5527,
             'permeability': 0.8468440619395835,
             'saturation': 16.0,
             'transmitted_bits': 197.76350000000002}}
Found implementation: anfext2l
  First time seen. Benchmarking.


...

{'basic': {'expected_sample_burst_size': 2.9887935251478632,
           'expected_sample_data_size': 16.47218392849202,
           'expected_time_to_recover': 33.607600000000005,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.14877587212416238,
             'bit_redundancy': 143.038,
             'ideal_bit_efficency': 0.17577049319160878,
             'ideal_bit_redundancy': 117.2309259424294,
             'ideal_packet_efficiency': 0.8788524659580439,
             'ideal_packet_redundancy': 23.44618518848588,
             'ideal_time_to_recover': 28.44618518848588,
             'ideal_transmitted_bits': 142.2309259424294,
             'packet_efficiency': 0.7438793606208118,
             'packet_redundancy': 28.607600000000005,
             'permeability': 0.8464212020044835,
             'saturation': 5.0,
             'transmitted_bits': 168.038}}
Found implementation: single_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3438.67it/s]


{'basic': {'expected_sample_burst_size': 2.9958638243716194,
           'expected_sample_data_size': 16.458399618199174,
           'expected_time_to_recover': 13.3227,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.34907945579761085,
             'bit_redundancy': 43.36009567122585,
             'ideal_bit_efficency': 0.4126211450089353,
             'ideal_bit_redundancy': 33.101934242812135,
             'ideal_packet_efficiency': 2.063105725044677,
             'ideal_packet_redundancy': 6.6203868485624255,
             'ideal_time_to_recover': 11.271067714317256,
             'ideal_transmitted_bits': 56.35533857158629,
             'packet_efficiency': 1.7453972789880545,
             'packet_redundancy': 8.672019134245168,
             'permeability': 0.8460047673757765,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.6135}}
Found implementation: single_binary_fullrank2
  First time s

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3420.82it/s]


{'basic': {'expected_sample_burst_size': 3.0105106282845715,
           'expected_sample_data_size': 16.37697074280336,
           'expected_time_to_recover': 13.8706,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3352905329080812,
             'bit_redundancy': 46.09959567122584,
             'ideal_bit_efficency': 0.3969256013670355,
             'ideal_bit_redundancy': 35.33038126905083,
             'ideal_packet_efficiency': 1.9846280068351771,
             'ideal_packet_redundancy': 7.0660762538101665,
             'ideal_time_to_recover': 11.716757119564997,
             'ideal_transmitted_bits': 58.583785597824985,
             'packet_efficiency': 1.6764526645404059,
             'packet_redundancy': 9.219919134245169,
             'permeability': 0.8447188383750521,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.353}}


In [11]:
# TODO: more datamining - plots of progression of ttr distributions wrt payload, erasure etc, comparing approaches in this way